<a href="https://colab.research.google.com/github/englissi/englissi/blob/main/%ED%95%9C%EA%B5%AD%EC%96%B4%EB%B0%9C%EC%9D%8C%ED%8F%89%EA%B0%80%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install gradio speechrecognition

!pip install gradio gtts

In [2]:
import gradio as gr
from gtts import gTTS
import speech_recognition as sr
from difflib import SequenceMatcher
import tempfile
import os

def tts_korean(word):
    tts = gTTS(text=word, lang='ko')
    temp_file_path = tempfile.mktemp(suffix=".mp3")
    tts.save(temp_file_path)
    return temp_file_path

def recognize_speech_from_microphone_korean(audio_path):
    recognizer = sr.Recognizer()
    try:
        with sr.AudioFile(audio_path) as source:
            audio_data = recognizer.record(source)
            text = recognizer.recognize_google(audio_data, language='ko-KR')
            return text
    except sr.UnknownValueError:
        return "Could not understand the audio"
    except sr.RequestError as e:
        return f"Could not request results from Google Speech Recognition service; {e}"
    except Exception as e:
        return str(e)

def calculate_similarity_korean(word, recognized_text):
    return SequenceMatcher(None, word, recognized_text).ratio() * 100

def process_audio_korean(word, audio_path):
    recognized_text = recognize_speech_from_microphone_korean(audio_path)
    if "Error" in recognized_text or "Could not" in recognized_text:
        return recognized_text, 0.0
    similarity = calculate_similarity_korean(word, recognized_text)
    return recognized_text, similarity

def evaluate_pronunciation_korean(word):
    temp_file_path = tts_korean(word)
    return temp_file_path

def process_all_korean(word, audio_path):
    recognized_text, similarity = process_audio_korean(word, audio_path)
    return recognized_text, similarity

with gr.Blocks() as demo:
    with gr.Row():
        word_input = gr.Textbox(label="발음 평가할 단어 입력")
        tts_button = gr.Button("단어 듣기")
    tts_audio = gr.Audio(label="원어민 발음", type="filepath")

    with gr.Row():
        mic_input = gr.Audio(label="당신의 발음", type="filepath")
        result_button = gr.Button("발음 평가하기")

    recognized_text = gr.Textbox(label="인식된 텍스트")
    similarity_score = gr.Number(label="유사도 (%)")

    tts_button.click(evaluate_pronunciation_korean, inputs=word_input, outputs=tts_audio)
    result_button.click(process_all_korean, inputs=[word_input, mic_input], outputs=[recognized_text, similarity_score])

demo.launch()


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://dfcb2421368b44db87.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
